In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import pandas as pd
import os
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print (device)

cpu


In [ ]:
# Downloading and preprocessing the data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
net.to(device)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(3): 
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
print('Finished Training')

[1,  2000] loss: 2.222
[1,  4000] loss: 1.918
[1,  6000] loss: 1.713
[1,  8000] loss: 1.622
[1, 10000] loss: 1.519
[1, 12000] loss: 1.495
[2,  2000] loss: 1.406
[2,  4000] loss: 1.373
[2,  6000] loss: 1.365
[2,  8000] loss: 1.315
[2, 10000] loss: 1.283
[2, 12000] loss: 1.295
[3,  2000] loss: 1.191
[3,  4000] loss: 1.214
[3,  6000] loss: 1.202
[3,  8000] loss: 1.188
[3, 10000] loss: 1.180
[3, 12000] loss: 1.159
Finished Training


In [ ]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
net = Net()
net.load_state_dict(torch.load('cifar_net.pth'))
net.to(device)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 56 %


In [ ]:
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with torch.no_grad():
    for data in test_loader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: plane is 68.1 %
Accuracy for class: car   is 62.6 %
Accuracy for class: bird  is 41.9 %
Accuracy for class: cat   is 21.6 %
Accuracy for class: deer  is 35.3 %
Accuracy for class: dog   is 45.5 %
Accuracy for class: frog  is 90.0 %
Accuracy for class: horse is 63.2 %
Accuracy for class: ship  is 64.4 %
Accuracy for class: truck is 76.8 %


In [ ]:
# Feature extraction algorithm
def extract_features(loader, model, dataset):
    features = []
    filenames = []

    model.eval()
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)
            features.append(outputs)
            filenames.extend(list(dataset.data[:len(filenames)]))
    features = torch.cat(features, dim=0)

    return features, filenames

In [ ]:
train_features, train_filenames = extract_features(train_loader, net, train_dataset)
test_features, test_filenames = extract_features(test_loader, net, test_dataset)

In [ ]:
# Creating DataFrame for train set
train_df = pd.DataFrame(train_features.cpu().numpy())

# Renam columns '0', '1', '2', ... to 'feature_Y' for train set
train_df.columns = [f'feature_{i}' for i in range(train_df.shape[1])]

# Adding a 'Case_Number' column with instance names for train set
train_df.insert(0, 'Case_Number', [f'Case_{i}' for i in range(train_df.shape[0])])

# Adding a 'target' column with CIFAR-10 target classes for train set
train_df['target'] = train_dataset.targets

# Get unique values from the 'target' column
unique_targets = sorted(train_df['target'].unique())

# Create new columns for each unique 'target' value
for target in unique_targets:
    train_df[target] = (train_df['target'] == target).astype(int)

# Cut out the column with the 'target' header
target_column = train_df.pop('target')

# Insert it at the last position
train_df['target'] = target_column

# Saving train set features to a CSV file
train_df.to_csv('./train_features.csv', index=False)

# Creating DataFrame for test set
test_df = pd.DataFrame(test_features.cpu().numpy())

# Rename columns '0', '1', '2', ... to 'feature_Y' for test set
test_df.columns = [f'feature_{i}' for i in range(test_df.shape[1])]

# Adding a 'Case_Number' column with instance names for test set
test_df.insert(0, 'Case_Number', [f'Case_{i}' for i in range(test_df.shape[0])])

# Adding a 'target' column with CIFAR-10 target classes for test set
test_df['target'] = test_dataset.targets

# Get unique values from the 'target' column
unique_targets = sorted(test_df['target'].unique())

# Create new columns for each unique 'target' value
for target in unique_targets:
    test_df[target] = (test_df['target'] == target).astype(int)

# Cut out the column with the 'target' header
target_column = test_df.pop('target')

# Insert it at the last position
test_df['target'] = target_column

# Saving test set features to a CSV file
test_df.to_csv('./test_features.csv', index=False)

print("Features are saved in train_features.csv and test_features.csv")

Features are saved in train_features.csv and test_features.csv


In [ ]:
# Specify the path to the CSV files
csv_train_file_path = "train_features.csv"
csv_test_file_path = "test_features.csv"

# Check if file exists
if not os.path.exists(csv_train_file_path):
    print(f"File {csv_train_file_path} does not exist.")
else:
    # Upload CSV file
    df = pd.read_csv(csv_train_file_path)

    # Get file size in bytes
    file_size_bytes = os.path.getsize(csv_train_file_path)

    # Convert size to megabytes
    file_size_mb = file_size_bytes / (1024 * 1024)

    # Get information about the number of rows and columns
    num_rows, num_columns = df.shape

    # Print information
    print(f"Train file size: {file_size_mb:.2f} MB")
    print(f"Number of lines in train file: {num_rows}")
    print(f"Number of columns in train file: {num_columns}")

# Check if file exists
if not os.path.exists(csv_test_file_path):
    print(f"File {csv_test_file_path} does not exist.")
else:
    # Upload CSV file
    df = pd.read_csv(csv_test_file_path)

    # Get file size in bytes
    file_size_bytes = os.path.getsize(csv_test_file_path)

    # Convert size to megabytes
    file_size_mb = file_size_bytes / (1024 * 1024)

    # Get information about the number of rows and columns
    num_rows, num_columns = df.shape

    # Print information
    print(f"\nTest file size: {file_size_mb:.2f} MB")
    print(f"Number of lines in test file: {num_rows}")
    print(f"Number of columns in test file: {num_columns}")
    print(f"\nFeatures from 1 to {num_columns-11}")

Train file size: 6.60 MB
Number of lines in train file: 50000
Number of columns in train file: 22

Test file size: 1.31 MB
Number of lines in test file: 10000
Number of columns in test file: 22

Features from 1 to 11


In [ ]:
#LDA
# Load training data from a CSV file
csv_train_file_path = "test_features.csv"
train_data = pd.read_csv('train_features.csv')

# Load test data from a separate CSV file
test_data = pd.read_csv('test_features.csv')

# Prepare training data
X_train = train_data.iloc[:, 1:11]  # Select features
y_train = train_data['target']  # Target variable

# Prepare test data
X_test = test_data.iloc[:, 1:11]  # Select features
y_test = test_data['target']  # Target variable

# Train the LDA model on the training data
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)

# Predict classes for the training data
y_train_pred = lda.predict(X_train)

# Predict classes for the test data
y_pred = lda.predict(X_test)

# Evaluate model accuracy on the training data
train_accuracy = accuracy_score(y_train, y_train_pred)
train_error_rate = 1 - train_accuracy

# Evaluate model accuracy on the test data
accuracy = accuracy_score(y_test, y_pred)
error_rate = 1 - accuracy

print(f'Accuracy on the training data: {train_accuracy * 100:.2f}%')
print(f'Error rate on the training data: {train_error_rate * 100:.2f}%')
print(f'Accuracy on the test data: {accuracy * 100:.2f}%')
print(f'Error rate on the test data: {error_rate * 100:.2f}%')

# Count the number of elements in each predicted class for the training data
predicted_class_counts_train = pd.Series(y_train_pred).value_counts().sort_index()
print("\nPredicted class counts in training data:")
print(predicted_class_counts_train)

# Count the number of elements in each predicted class for the test data
predicted_class_counts_test = pd.Series(y_pred).value_counts().sort_index()
print("\nPredicted class counts in test data:")
print(predicted_class_counts_test)

Accuracy on the training data: 10.54%
Error rate on the training data: 89.46%
Accuracy on the test data: 9.23%
Error rate on the test data: 90.77%

Predicted class counts in training data:
0     274
1    3370
2    4412
3    5983
4    5108
5    2799
6    7457
7    4803
8    8813
9    6981
dtype: int64

Predicted class counts in test data:
0      53
1     695
2     838
3    1181
4    1017
5     581
6    1504
7     966
8    1783
9    1382
dtype: int64


In [ ]:
#Logistic Regression
# Load training data from a CSV file
train_data = pd.read_csv('train_features.csv')

# Load test data from a separate CSV file
test_data = pd.read_csv('test_features.csv')

# Prepare training data
X_train = train_data.iloc[:, 1:11]  # Select features
y_train = train_data['target']  # Target variable

# Prepare test data
X_test = test_data.iloc[:, 1:11]  # Select features
y_test = test_data['target']  # Target variable

# Train the Logistic Regression model on the training data
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)

# Predict classes for the training data
y_train_pred = lr.predict(X_train)

# Predict classes for the test data
y_pred = lr.predict(X_test)

# Evaluate model accuracy on the training data
train_accuracy = accuracy_score(y_train, y_train_pred)
train_error_rate = 1 - train_accuracy

# Evaluate model accuracy on the test data
accuracy = accuracy_score(y_test, y_pred)
error_rate = 1 - accuracy

print(f'Accuracy on the training data: {train_accuracy * 100:.2f}%')
print(f'Error rate on the training data: {train_error_rate * 100:.2f}%')
print(f'Accuracy on the test data: {accuracy * 100:.2f}%')
print(f'Error rate on the test data: {error_rate * 100:.2f}%')

# Count the number of elements in each predicted class for the training data
predicted_class_counts_train = pd.Series(y_train_pred).value_counts().sort_index()
print("\nPredicted class counts in training data:")
print(predicted_class_counts_train)

# Count the number of elements in each predicted class for the test data
predicted_class_counts_test = pd.Series(y_pred).value_counts().sort_index()
print("\nPredicted class counts in test data:")
print(predicted_class_counts_test)

Accuracy on the training data: 10.53%
Error rate on the training data: 89.47%
Accuracy on the test data: 9.19%
Error rate on the test data: 90.81%

Predicted class counts in training data:
0     282
1    3386
2    4360
3    5943
4    5130
5    2906
6    7491
7    4878
8    8732
9    6892
dtype: int64

Predicted class counts in test data:
0      56
1     696
2     832
3    1178
4    1022
5     596
6    1507
7     966
8    1775
9    1372
dtype: int64
